# Cálculo de las tasas de distribución para la rama Q mediante Python


---
El objetivo de este documento es presentar las funciones y sintaxis necesarias para el cáculo de las tasas de distribución usando el lenguaje de programación Python, bajo la premisa de que esta herramienta permitirá realizar cálculos más confiables y transparentes, ayudará a la acumulación de capital humano de los funcionarios, y  permitirá una transición más suave al análisis de la información una vez calculada.


In [ ]:
import pandas as pd
import requests
import xlrd
import urllib

link = "https://www.dssa.gov.co/images/estadisticas/aseguramiento/Estad%C3%ADsticas_Cobertura_en_Aseguramiento_Diciembre_2018-Ajustado_CENS_DANE.xlsx"


In [ ]:
#Se lee el archivo 
afiliados=pd.ExcelFile(link)
#Se selecciona la primera hoja
afiliados.sheet_names
#Se asigna la primera hoja mediante el método .parse; y se eliminan las filas iniciales.
datos_afiliacion=afiliados.parse(afiliados.sheet_names[1],header=[0,1],skiprows=[0,1])
#¿Cómo se ven los datos?
datos_afiliacion.head()




,COD MPIO,MUNICIPIO,Unnamed: 2_level_0,REGIMEN SUBSIDIADO,Unnamed: 4_level_0,REGIMEN CONTRIBUTIVO,Unnamed: 6_level_0,"REGIMEN ESPECIAL Y EXCEPCIÓN\n(Magisterio, Ecopetrol, \nUdeA, Unal)",Unnamed: 8_level_0,FUERZAS MILITARES\n(Ejercito y Policia),Unnamed: 10_level_0,Total afiliados al SGSSS,Unnamed: 12_level_0,Unnamed: 13_level_0,Unnamed: 14_level_0,Unnamed: 15_level_0,Unnamed: 16_level_0,Unnamed: 17_level_0,Unnamed: 18_level_0,Unnamed: 19_level_0,Unnamed: 20_level_0,Unnamed: 21_level_0,Unnamed: 22_level_0,Unnamed: 23_level_0,Unnamed: 24_level_0,Unnamed: 25_level_0,Unnamed: 26_level_0,Unnamed: 27_level_0,Unnamed: 28_level_0,Unnamed: 29_level_0,Unnamed: 30_level_0,Unnamed: 31_level_0,Unnamed: 32_level_0
,Unnamed: 0_level_1,Unnamed: 1_level_1,Población proyectada DANE 2018,Total Afiliados,% de cobertura,Total Afiliados,% de cobertura,Total Afiliados,% de cobertura,Total Afiliados,% de cobertura,Total afiliados al SGSSS,% de Cobertura\nRS+RC+RE,% de Cobertura\nRS+RC+RE,Unnamed: 14_level_1,Unnamed: 15_level_1,Unnamed: 16_level_1,Unnamed: 17_level_1,,Unnamed: 19_level_1,Unnamed: 20_level_1,Unnamed: 21_level_1,Unnamed: 22_level_1,Unnamed: 23_level_1,Unnamed: 24_level_1,Unnamed: 25_level_1,Unnamed: 26_level_1,Unnamed: 27_level_1,Unnamed: 28_level_1,Unnamed: 29_level_1,Unnamed: 30_level_1,Unnamed: 31_level_1,Unnamed: 32_level_1
0,NaN,TOTAL DEPARTAMENTO,6407102.0,2338345.0,0.364961,3791942.0,0.591834,104367.0,0.016289,82194.0,0.012829,6316848.0,0.985913,0.944077,22431.0,177353.0,NaN,NaN,NaN,NaN,R. excepción,NaN,Ejercito y policia,NaN,NaN,Ejercito,Policia,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,TOTAL MAGDALENA MEDIO,103592.0,63304.0,61.108966,26341.0,25.427639,1921.0,1.854390,2018.0,1.948027,93584.0,90.339022,77.466992,685.0,5684.0,Campo20,CuentaDeCampo6,Campo9,CuentaDeCampo2,1,47571.0,1,43397.0,MEDELLIN,12529,30868,NaN,NaN,NaN,NaN,NaN,NaN
2,142,CARACOLI,4582.0,3027.0,66.062855,883.0,19.271061,79.0,0.017241,44.0,0.009603,4033.0,88.018333,89.245408,14.0,135.0,1,607975,1,1999006,2,463.0,2,78.0,ABEJORRAL,14,64,NaN,NaN,NaN,NaN,NaN,NaN
3,425,MACEO,8221.0,6117.0,74.407006,1800.0,21.895147,170.0,0.020679,38.0,0.004622,8125.0,98.832259,100.000000,36.0,290.0,2,12615,2,2464,4,41.0,4,3.0,ABRIAQUI,3,0,NaN,NaN,NaN,NaN,NaN,NaN
4,579,PUERTO BERRIO,39314.0,27241.0,69.290838,16642.0,42.330976,911.0,0.023172,1615.0,0.041080,46409.0,100.000000,93.960560,305.0,2389.0,4,1392,4,309,21,55.0,21,32.0,ALEJANDRIA,12,20,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Se eliminan las notas de pie de página.
datos_afiliacion=afiliados.parse(afiliados.sheet_names[1],header=[0],skiprows=[0,1,2],usecols=list(range(0,11)),skipfooter=4)
#Se renombran las columnas.
datos_afiliacion.columns=["codmun","municipio","poblacion","regsub_afiliados","regsub_cobertura","regcon_afiliados","regcon_cobertura", "regesp_afiliados","regesp_cobertura","regmil_afiliados","regmil_cobertura"]
#¿Cómo se ven los datos?
datos_afiliacion.head()



,codmun,municipio,poblacion,regsub_afiliados,regsub_cobertura,regcon_afiliados,regcon_cobertura,regesp_afiliados,regesp_cobertura,regmil_afiliados,regmil_cobertura
0,NaN,TOTAL DEPARTAMENTO,6407102,2338345,0.364961,3791942,0.591834,104367,0.016289,82194,0.012829
1,NaN,TOTAL MAGDALENA MEDIO,103592,63304,61.108966,26341,25.427639,1921,1.854390,2018,1.948027
2,142.0,CARACOLI,4582,3027,66.062855,883,19.271061,79,0.017241,44,0.009603
3,425.0,MACEO,8221,6117,74.407006,1800,21.895147,170,0.020679,38,0.004622
4,579.0,PUERTO BERRIO,39314,27241,69.290838,16642,42.330976,911,0.023172,1615,0.041080


In [ ]:
#Eliminar las columnas de los totales por subregión.
datos_afiliacion=datos_afiliacion.loc[-datos_afiliacion["municipio"].str.contains("TOTAL",regex=False),:]
#Cambiar el tipo a la columna de código de municipio.
datos_afiliacion.codmun=datos_afiliacion.codmun.astype("int")
#Esto se hace para sumarle 5000 y obtener el código del DANE.
datos_afiliacion["codmun_dane"]=datos_afiliacion.codmun+5000
#Usar el código del DANE como índice
datos_afiliacion=datos_afiliacion.set_index("codmun_dane")
datos_afiliacion=datos_afiliacion.drop("codmun",axis=1)

In [ ]:
#Ordenar de mayor a menor el código
datos_afiliacion=datos_afiliacion.sort_values(by="codmun_dane")
#Se agregan los datos de afiliación
datos_afiliacion["total_afiliados"]=datos_afiliacion.iloc[:,[2,4,6,8]].sum(axis=1)
datos_afiliacion
#Se crea la variable participación
datos_afiliacion["participacion"]=datos_afiliacion.total_afiliados/datos_afiliacion.total_afiliados.sum(axis=0)



In [ ]:
valor_ag_precios_corrientes=float(input())

3300


In [ ]:
valor_ag_precios_constantes=float(input())

2968.79


In [ ]:
# Crear un nuevo dataframe para la participación.
valor_distribuido=datos_afiliacion.loc[:,["participacion"]]
#Aplicar el valor del DANE a la participación.
valor_distribuido["valor_precios_corr"]=valor_distribuido["participacion"]*valor_ag_precios_corrientes
valor_distribuido["valor_precios_con"]=valor_distribuido["participacion"]*valor_ag_precios_constantes

In [ ]:
from google.colab import files

valor_distribuido.to_csv('distribucion_q.csv')
files.download('distribucion_q.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>